In [1]:
import numpy as np
import scipy.linalg as linalg
import matplotlib.pyplot as plt

In [2]:
class Material(object):
    def __init__(self, D, den):
        self.D = D # elasticity matrix
        self.den = den # mass density in Mg per cubic meter

In [3]:
"""
%Coefficient matrices of a 2-node line element
%
%  Inputs:
%   xy(i,:)    - coordinates of node i (orgin at scaling centre).
%                The nodes are numbered locally within each line element
%   mat        - material constants
%     mat.D    - elasticity matrix
%     mat.den  - mass density
%
%  Outputs:
%    e0, e1, e2, m0  - element coefficient matrices
"""
def EleCoeff2NodeEle(xy, mat:Material):
    dxy = xy[1, :] - xy[0, :] # (2.50a), (2.50b)
    mxy = np.sum(xy, axis=0)/2 # (2.51a), (2.51b)
    a = xy[0,0]*xy[1,1]-xy[1,0]*xy[0,1] # a=2|J_b| (2.58)
    if a < 1e-10:
        raise ValueError('negative area (EleCoeff2NodeEle)')
    
    C1 = 0.5*np.array([ [dxy[1], 0], [0, -dxy[0]], [-dxy[0], dxy[1]] ]) # (2.114a)
    C2 = np.array([ [-mxy[1], 0], [0, mxy[0]], [mxy[0], -mxy[1]] ]) # (2.114b)
    
    Q0 = 1/a*(np.matmul(np.matmul(C1.T, mat.D), C1)) # (2.118a)
    Q1 = 1/a*(np.matmul(np.matmul(C2.T, mat.D), C1)) # (2.118b)
    Q2 = 1/a*(np.matmul(np.matmul(C2.T, mat.D), C2)) # (2.118c)
    
    e0 =  2/3*np.block([ [2*Q0, Q0], [Q0, 2*Q0] ]) # (2.119a)
    e1 = -1/3*np.block([ [Q0, -Q0], [-Q0, Q0] ]) + np.block([ [-Q1, -Q1], [Q1, Q1] ]) # (2.119b)
    e2 =  1/3*np.block([ [Q0, -Q0], [-Q0, Q0] ]) + np.block([ [Q2, -Q2], [-Q2, Q2] ]) # (2.119c)
    
    # mass coefficent matrix
    m0 = a*mat.den/6*np.array([ [2, 0, 1, 0], [0, 2, 0, 1], [1, 0, 2, 0], [0, 1, 0, 2] ]) # (3.112)
    
#     print('dxy: ', dxy)
#     print('mxy: ', mxy)
#     print('a: ', a)
#     print('C1: ', C1)
#     print('C2: ', C2)
#     print('Q0: ', Q0)
#     print('Q1: ', Q1)
#     print('Q2: ', Q2)
#     print('e0: ', e0)
#     print('e1: ', e1)
#     print('e2: ', e2)
#     print('m0: ', m0)
    
    return e0, e1, e2, m0


xy = np.array([[1, -1], [1, 1]])
E = 12
D = E/2*np.array([[2, 0, 0], [0, 2, 0], [0, 0, 1]])
mat = Material(D, 1)
EleCoeff2NodeEle(xy, mat)

(array([[8., 0., 4., 0.],
        [0., 4., 0., 2.],
        [4., 0., 8., 0.],
        [0., 2., 0., 4.]]), array([[-2., -3.,  2., -3.],
        [-0., -1.,  0.,  1.],
        [ 2.,  3., -2.,  3.],
        [ 0.,  1.,  0., -1.]]), array([[ 5.,  0., -5., -0.],
        [ 0.,  7., -0., -7.],
        [-5., -0.,  5.,  0.],
        [-0., -7.,  0.,  7.]]), array([[0.66666667, 0.        , 0.33333333, 0.        ],
        [0.        , 0.66666667, 0.        , 0.33333333],
        [0.33333333, 0.        , 0.66666667, 0.        ],
        [0.        , 0.33333333, 0.        , 0.66666667]]))